In [ ]:

#Function to detect edges
def line_detection(image)
    
#function to locate the intersections of lines 
def intersections(lines)

#project the point on image to the real system
def pinhole_model(x,y)

#Function to locate the start point to robot coordinate system
def locate_start_point(x, y)




In [1]:
import cv2 as cv
import numpy as np

#detect the line using SHT
def line_detection(image):
    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    #load camera parameters
    # camMtx = np.genfromtxt("intrinsics.csv", delimiter = ",") 
    # dist = np.genfromtxt("distortion.csv", delimiter = ",") 
    
    #image processing
    #img = cv.undistort(img, camMtx, dist, None)
    img = cv.GaussianBlur(img,(3,3),0)
    nimg = cv.bitwise_not(img)
    
    # compute the median of the single channel pixel intensities
    v = np.median(img)
    nv = np.median(nimg)
    sigma = 0.33
    
    #Canny edge detection
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edges = cv.Canny(img, lower, upper)
    nlower = int(max(0, (1.0 - sigma) * nv))
    nupper = int(min(255, (1.0 + sigma) * nv))
    nedges = cv.Canny(nimg, nlower, nupper)
    
    cv.imwrite("edges.png",edges)
    cv.imwrite("negative_edges.png",nedges)
    
    # run Hough on edge detected image
    if nedges.mean() > edges.mean():
        lines = cv.HoughLines(nedges,1,np.pi/180,135)
    else:
        lines = cv.HoughLines(edges,1,np.pi/180,135)
    # clean noisy lines due to distortion correction
    height, width = img.shape
    alpha = 0.05
    c = 0
    for i,line in enumerate(lines):
        if (abs(line[0][0]) > (1-alpha)*width) | (abs(line[0][0]) < alpha*height) | ((abs(line[0][0]) > (1-alpha)*height) & (abs(line[0][0]) < height) & (abs(line[0][1]) > np.pi/4)):
            lines = np.delete(lines,i-c,axis=0)
            c = c+1
    res_img = img.copy()
    
    #plot the lines
    for line in lines:
        rho,theta=line[0]
        a=np.cos(theta)
        b=np.sin(theta)
        x0=a*rho
        y0=b*rho
        x1=int(x0+100000*(-b))
        y1=int(y0+100000*a)
        x2=int(x0-100000*(-b))
        y2 = int(y0 - 100000 * a)
        cv.line(image,(x1,y1),(x2,y2),(0,0,255),2)
        print("Line",  ":(", x1,y1,"),(",x2,y2,")")
             
     
    cv.imshow("img lines",image)
    cv.imwrite("linesDetected.jpg",res_img)
    images = np.hstack((img,nimg,edges,nedges,res_img))

    cv.namedWindow("image", cv.WINDOW_AUTOSIZE)
    cv.startWindowThread()

    
#src = cv.imread("IMG_3059.jpeg") 
#line_detection(src)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [ ]:
import cv2 as cv
import numpy as np

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])
#revise the line representation 

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y
    
    
lineA = [[-93621, 35157],[94317, -33246]]
lineB =  [[-93513, -35431],[93202, 36241]]

[a,b] = line_intersection(lineA, lineB)
x = np.round(a).astype("int")
y = np.round(b).astype("int")
print(x,y)

src = cv.imread("IMG_3059.jpeg")
line_detection(src)
cv.circle(src,(x,y), 20,(0,0,255),-1)

cv.imshow("img",src)
cv.waitKey(0)
cv.destroyAllWindows()

826 782
Line :( -93621 35157 ),( 94317 -33246 )
Line :( -93513 -35431 ),( 93202 36241 )
Line :( -92871 -37080 ),( 92564 37840 )


In [7]:
import cv2 as cv
import numpy as np

# project the point on image to 3d camera space
def pinhole_model(x, y):
  
    # location of camera, unit: mm, origin point
    L = [0, 0, 0]
    
    # reference point in camera system, unit: mm, fixed value
    r0 = [500,500,-500]
    # reference point: on image, unit: pixel, fixed value
    r = [900,900]
    
    # focalLength is fixed
    f = 250 #assume vaule
    
    # Calculate the real coordinate using pinhole model
    # W = (DxP)/F
    x_real = (L[2]-r0[2])*(x-r[0])/f
    y_real = (L[2]-r0[2])*(y-r[1])/f
    
    position = [x_real, y_real, -500]
    return position
    print (position)

a = 826 
b = 782
pinhole_model(a, b)

[-148.0, -236.0, -500]

In [2]:
import cv2 as cv
import numpy as np

#projecct the point from camera into robot coordinate system 
def locate_start_point(x,y,z):
    
    # Define the origin point in robot space
    RL = [0,0,0]
    
    #Assume the distance between the two origin point
    LinR = [500,500,600]
    
    #calculate the start point 
    SPoint = [x+LinR[0], y+LinR[1], z+LinR[2]]
    
    print(SPoint)

locate_start_point(-148.0, -236.0, -500)
    


[352.0, 264.0, 100]
